In [ ]:
# --- PESV v3 "Championship" Ablation Study (MULTI-MODEL) ---
#
# This script loads the final 'final_PESV_dataset_v3.csv'
# and runs a "championship" ablation study comparing:
# 1. Random Forest (Bagging)
# 2. XGBoost (Boosting)
# 3. SVM (Geometric/Hyperplane)
#
# It iterates through all Tasks -> Models -> Feature Sets automatically.

print("--- Initializing PESV v3 Championship (Multi-Model) ---")

import pandas as pd
import numpy as np
import time
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Try importing XGBoost; handle if missing
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    print("WARNING: XGBoost not installed. Skipping XGBoost.")
    HAS_XGB = False

print("All libraries imported successfully.")

# --- PART 1: Configuration ---

# --- !! SET YOUR EXPERIMENT MODE !! ---
# "FULL"        - Runs on the full v3 dataset
# "VPN_ONLY"    - Runs on only the VPN samples
# "NONVPN_ONLY" - Runs on only the NonVPN samples
EXPERIMENT_MODE = "VPN_ONLY"

BASE_PATH = "/content/drive/MyDrive/1 Skripsi/"
FINAL_PESV_FILE = os.path.join(BASE_PATH, "VPNOnly-final_PESV_dataset_v3.csv")

# --- DEFINE THE CONTENDERS ---
MODELS = {
    "Random Forest": RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1
    ),
    "SVM": SVC(
        kernel='rbf',
        class_weight='balanced',
        random_state=42,
        probability=False # Set True only if you need predict_proba (slower)
    )
}

# Add XGBoost if installed
if HAS_XGB:
    MODELS["XGBoost"] = XGBClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        # XGBoost handles multiclass automatically.
        # We use default weights here for stability across different tasks.
    )

TEST_SET_SIZE = 0.2
RANDOM_STATE = 42

# --- PART 2: Load Data and Define Feature Sets ---

def load_data(experiment_mode):
    """Loads and filters the dataset based on the experiment mode."""
    print(f"\n--- Loading Data for Mode: {experiment_mode} ---")
    if not os.path.exists(FINAL_PESV_FILE):
        print(f"FATAL ERROR: Could not find dataset at '{FINAL_PESV_FILE}'")
        return None, None

    df_full = pd.read_csv(FINAL_PESV_FILE)

    if experiment_mode == "FULL":
        df = df_full
    elif experiment_mode == "VPN_ONLY":
        df = df_full[df_full['binary_type'] == 'VPN'].copy()
    elif experiment_mode == "NONVPN_ONLY":
        df = df_full[df_full['binary_type'] == 'NonVPN'].copy()
    else:
        print(f"FATAL ERROR: Unknown experiment mode '{experiment_mode}'")
        return None, None

    if df.empty:
        print("FATAL ERROR: The filtered dataset is empty.")
        return None, None

    print(f"Loaded dataset with shape: {df.shape}")

    # --- Define Feature Column Groups ---
    all_cols = set(df.columns)

    # 1. Find Alpha'' (α'') columns
    alpha_pp_cols = sorted(list([c for c in all_cols if c.startswith('alpha_pp_')]))

    # 2. Find Delta (δ) columns
    delta_cols_set = set([c for c in all_cols if
                          c.startswith('c2s_') or
                          c.startswith('s2c_') or
                          c.startswith('flow_') or
                          c.startswith('total_')])
    delta_cols = sorted(list(delta_cols_set))

    # 3. Find Gamma' (γ') columns
    gamma_p_cols_set = set([c for c in all_cols if c.startswith('burst_')])
    gamma_p_cols = sorted(list(gamma_p_cols_set))

    # Master list of feature sets
    feature_sets = {
        "Alpha'' (α'') only": alpha_pp_cols,
        "Delta (δ) only": delta_cols,
        "Gamma' (γ') only": gamma_p_cols,
        "Alpha'' + Delta": alpha_pp_cols + delta_cols,
        "Alpha'' + Gamma'": alpha_pp_cols + gamma_p_cols,
        "Delta + Gamma'": delta_cols + gamma_p_cols,
        "Full (α'' + δ + γ')": alpha_pp_cols + delta_cols + gamma_p_cols,
    }

    print(f"Found {len(alpha_pp_cols)} Alpha'' features.")
    print(f"Found {len(delta_cols)} Delta features.")
    print(f"Found {len(gamma_p_cols)} Gamma' features.")

    if not alpha_pp_cols or not delta_cols or not gamma_p_cols:
        print("FATAL ERROR: Could not find all feature columns.")
        return None, None

    return df, feature_sets

# --- PART 3: Classification Task Function (UPDATED) ---

def run_classification_task(df, target_label, feature_set_name, feature_cols, model_name, model_instance):
    """
    Runs a classification pipeline for a specific Model + Feature Set combination.
    Handles Label Encoding automatically for XGBoost.
    """
    print(f"\nRunning: {model_name} | Target: {target_label} | Features: {feature_set_name}")

    # --- 1. Prepare Data ---
    # --- 1. Prepare Data & Split (FIXED) ---
    SPLIT_MAP_FILE = os.path.join(BASE_PATH, "train_test_split_map.csv")
    if not os.path.exists(SPLIT_MAP_FILE):
        print("CRITICAL ERROR: Split map not found! Please run alpha2.ipynb first.")
        return {}

    # Load split map
    df_split = pd.read_csv(SPLIT_MAP_FILE)
    
    # Ensure df has 'filename'
    if 'filename' not in df.columns:
         print("Error: 'filename' column missing from dataframe.")
         return {}

    df_merged = pd.merge(df, df_split, on='filename', how='inner')
    
    # Separate into Train and Test DataFrames based on the map
    df_train_set = df_merged[df_merged['split_group'] == 'TRAIN']
    df_test_set = df_merged[df_merged['split_group'] == 'TEST']
    
    # Extract Features (X) and Target (y) for each set
    X_train = df_train_set[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    y_train_raw = df_train_set[target_label]
    
    X_test = df_test_set[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    y_test_raw = df_test_set[target_label]

    print(f"   > Split Stats: Train={len(X_train)}, Test={len(X_test)}")

    # Encode Labels (Fit on ALL to ensure all classes are known)
    le = LabelEncoder()
    le.fit(pd.concat([y_train_raw, y_test_raw])) 
    
    y_train = le.transform(y_train_raw)
    y_test = le.transform(y_test_raw)
    class_labels = [str(c) for c in le.classes_]

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model_instance)
    ])

    # --- 4. Train ---
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    end_time = time.time()
    train_time = end_time - start_time

    # --- 5. Evaluate ---
    y_pred = pipeline.predict(X_test)

    # We pass target_names=class_labels so the report shows 'Chat', 'VoIP' etc.
    # instead of 0, 1, 2.
    report_dict = classification_report(
        y_test, y_pred, target_names=class_labels, zero_division=0, output_dict=True
    )

    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': report_dict['weighted avg']['precision'],
        'recall': report_dict['weighted avg']['recall'],
        'f1_score': report_dict['weighted avg']['f1-score'],
        'time': train_time,
        'model_name': model_name
    }

    print(f" > Acc: {metrics['accuracy']:.4f} | F1: {metrics['f1_score']:.4f} | Time: {train_time:.2f}s")
    return metrics

# --- PART 4: Main Execution ---
def main():
    warnings.filterwarnings("ignore")
    df, feature_sets = load_data(EXPERIMENT_MODE)
    if df is None: return

    # Define tasks based on mode
    if EXPERIMENT_MODE == "FULL":
        tasks_to_run = ['binary_type', 'category', 'application']
    else:
        tasks_to_run = ['category', 'application']

    # Storage: summary_results[task][model_name][feature_set] = metrics
    summary_results = {}

    # --- THE CHAMPIONSHIP LOOP ---
    for task in tasks_to_run:
        print(f"\n{'='*80}")
        print(f"--- STARTING TOURNAMENT FOR TARGET: {task} ---")
        print(f"{'='*80}")

        task_results = [] # List to store flattened results for sorting

        for model_name, model_inst in MODELS.items():
            print(f"\n--- Evaluating Contender: {model_name} ---")

            for set_name, cols in feature_sets.items():
                if not cols: continue

                # Run the task
                metrics = run_classification_task(df, task, set_name, cols, model_name, model_inst)

                # Save results flatly for easy sorting
                metrics['feature_set'] = set_name
                task_results.append(metrics)

        summary_results[task] = task_results

    # --- Final Summary Report ---
    print(f"\n{'='*80}")
    print(f"--- FINAL CHAMPIONSHIP STANDINGS ({EXPERIMENT_MODE}) ---")
    print(f"{'='*80}\n")

    for task, results_list in summary_results.items():
        print(f"--- Target: {task} ---")

        # Sort by Accuracy (Desc)
        sorted_results = sorted(results_list, key=lambda x: x['accuracy'], reverse=True)

        # Define Header
        # M = Model, FS = Feature Set, Acc = Accuracy, F1 = F1-Score, T = Time
        header = f" {'Model':<15} | {'Feature Set':<25} | {'Acc %':<8} | {'F1 %':<8} | {'Time (s)':<8}"
        print(header)
        print("-" * len(header))

        for res in sorted_results:
            print(f" {res['model_name']:<15} | {res['feature_set']:<25} | "
                  f"{res['accuracy']*100:6.2f} | {res['f1_score']*100:6.2f} | "
                  f"{res['time']:6.2f}")
        print("\n")

    print("--- Championship Finished ---")

if __name__ == "__main__":
    if not os.path.exists("/content/drive/MyDrive"):
        print("Please mount your Google Drive first!")
    else:
        main()

--- Initializing PESV v3 Championship (Multi-Model) ---
All libraries imported successfully.

--- Loading Data for Mode: VPN_ONLY ---
Loaded dataset with shape: (2623, 208)
Found 128 Alpha'' features.
Found 40 Delta features.
Found 36 Gamma' features.

--- STARTING TOURNAMENT FOR TARGET: category ---

--- Evaluating Contender: Random Forest ---

Running: Random Forest | Target: category | Features: Alpha'' (α'') only
 > Acc: 0.9048 | F1: 0.9036 | Time: 1.38s

Running: Random Forest | Target: category | Features: Delta (δ) only
 > Acc: 0.9314 | F1: 0.9310 | Time: 0.98s

Running: Random Forest | Target: category | Features: Gamma' (γ') only
 > Acc: 0.9029 | F1: 0.9028 | Time: 0.96s

Running: Random Forest | Target: category | Features: Alpha'' + Delta
 > Acc: 0.9333 | F1: 0.9328 | Time: 0.82s

Running: Random Forest | Target: category | Features: Alpha'' + Gamma'
 > Acc: 0.9448 | F1: 0.9442 | Time: 0.86s

Running: Random Forest | Target: category | Features: Delta + Gamma'
 > Acc: 0.9333

In [ ]:
# --- PESV v3 "Championship" Ablation Study (MULTI-MODEL) ---
#
# This script loads the final 'final_PESV_dataset_v3.csv'
# and runs a "championship" ablation study comparing:
# 1. Random Forest (Bagging)
# 2. XGBoost (Boosting)
# 3. SVM (Geometric/Hyperplane)
#
# It iterates through all Tasks -> Models -> Feature Sets automatically.

print("--- Initializing PESV v3 Championship (Multi-Model) ---")

import pandas as pd
import numpy as np
import time
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Try importing XGBoost; handle if missing
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    print("WARNING: XGBoost not installed. Skipping XGBoost.")
    HAS_XGB = False

print("All libraries imported successfully.")

# --- PART 1: Configuration ---

# --- !! SET YOUR EXPERIMENT MODE !! ---
# "FULL"        - Runs on the full v3 dataset
# "VPN_ONLY"    - Runs on only the VPN samples
# "NONVPN_ONLY" - Runs on only the NonVPN samples
EXPERIMENT_MODE = "FULL"

BASE_PATH = "/content/drive/MyDrive/1 Skripsi/"
FINAL_PESV_FILE = os.path.join(BASE_PATH, "final_PESV_dataset_v3.csv")

# --- DEFINE THE CONTENDERS ---
MODELS = {
    "Random Forest": RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1
    ),
    "SVM": SVC(
        kernel='rbf',
        class_weight='balanced',
        random_state=42,
        probability=False # Set True only if you need predict_proba (slower)
    )
}

# Add XGBoost if installed
if HAS_XGB:
    MODELS["XGBoost"] = XGBClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        # XGBoost handles multiclass automatically.
        # We use default weights here for stability across different tasks.
    )

TEST_SET_SIZE = 0.2
RANDOM_STATE = 42

# --- PART 2: Load Data and Define Feature Sets ---

def load_data(experiment_mode):
    """Loads and filters the dataset based on the experiment mode."""
    print(f"\n--- Loading Data for Mode: {experiment_mode} ---")
    if not os.path.exists(FINAL_PESV_FILE):
        print(f"FATAL ERROR: Could not find dataset at '{FINAL_PESV_FILE}'")
        return None, None

    df_full = pd.read_csv(FINAL_PESV_FILE)

    if experiment_mode == "FULL":
        df = df_full
    elif experiment_mode == "VPN_ONLY":
        df = df_full[df_full['binary_type'] == 'VPN'].copy()
    elif experiment_mode == "NONVPN_ONLY":
        df = df_full[df_full['binary_type'] == 'NonVPN'].copy()
    else:
        print(f"FATAL ERROR: Unknown experiment mode '{experiment_mode}'")
        return None, None

    if df.empty:
        print("FATAL ERROR: The filtered dataset is empty.")
        return None, None

    print(f"Loaded dataset with shape: {df.shape}")

    # --- Define Feature Column Groups ---
    all_cols = set(df.columns)

    # 1. Find Alpha'' (α'') columns
    alpha_pp_cols = sorted(list([c for c in all_cols if c.startswith('alpha_pp_')]))

    # 2. Find Delta (δ) columns
    delta_cols_set = set([c for c in all_cols if
                          c.startswith('c2s_') or
                          c.startswith('s2c_') or
                          c.startswith('flow_') or
                          c.startswith('total_')])
    delta_cols = sorted(list(delta_cols_set))

    # 3. Find Gamma' (γ') columns
    gamma_p_cols_set = set([c for c in all_cols if c.startswith('burst_')])
    gamma_p_cols = sorted(list(gamma_p_cols_set))

    # Master list of feature sets
    feature_sets = {
        "Alpha'' (α'') only": alpha_pp_cols,
        "Delta (δ) only": delta_cols,
        "Gamma' (γ') only": gamma_p_cols,
        "Alpha'' + Delta": alpha_pp_cols + delta_cols,
        "Alpha'' + Gamma'": alpha_pp_cols + gamma_p_cols,
        "Delta + Gamma'": delta_cols + gamma_p_cols,
        "Full (α'' + δ + γ')": alpha_pp_cols + delta_cols + gamma_p_cols,
    }

    print(f"Found {len(alpha_pp_cols)} Alpha'' features.")
    print(f"Found {len(delta_cols)} Delta features.")
    print(f"Found {len(gamma_p_cols)} Gamma' features.")

    if not alpha_pp_cols or not delta_cols or not gamma_p_cols:
        print("FATAL ERROR: Could not find all feature columns.")
        return None, None

    return df, feature_sets

# --- PART 3: Classification Task Function (UPDATED) ---

def run_classification_task(df, target_label, feature_set_name, feature_cols, model_name, model_instance):
    """
    Runs a classification pipeline for a specific Model + Feature Set combination.
    Handles Label Encoding automatically for XGBoost.
    """
    print(f"\nRunning: {model_name} | Target: {target_label} | Features: {feature_set_name}")

    # --- 1. Prepare Data ---
    X = df[feature_cols]
    y = df[target_label]

    # Handle NaN/Inf
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

    # --- FIX: Encode Target Labels (Required for XGBoost) ---
    # XGBoost crashes if y contains strings like 'Chat', 'VoIP'.
    # We encode them to 0, 1, 2... here.
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_labels = [str(c) for c in le.classes_] # Save original names for report

    # --- 2. Train/Test Split ---
    # Note: We use y_encoded here instead of y
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=TEST_SET_SIZE, random_state=RANDOM_STATE, stratify=y_encoded
    )

    # --- 3. Create Pipeline ---
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model_instance)
    ])

    # --- 4. Train ---
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    end_time = time.time()
    train_time = end_time - start_time

    # --- 5. Evaluate ---
    y_pred = pipeline.predict(X_test)

    # We pass target_names=class_labels so the report shows 'Chat', 'VoIP' etc.
    # instead of 0, 1, 2.
    report_dict = classification_report(
        y_test, y_pred, target_names=class_labels, zero_division=0, output_dict=True
    )

    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': report_dict['weighted avg']['precision'],
        'recall': report_dict['weighted avg']['recall'],
        'f1_score': report_dict['weighted avg']['f1-score'],
        'time': train_time,
        'model_name': model_name
    }

    print(f" > Acc: {metrics['accuracy']:.4f} | F1: {metrics['f1_score']:.4f} | Time: {train_time:.2f}s")
    return metrics

# --- PART 4: Main Execution ---
def main():
    warnings.filterwarnings("ignore")
    df, feature_sets = load_data(EXPERIMENT_MODE)
    if df is None: return

    # Define tasks based on mode
    if EXPERIMENT_MODE == "FULL":
        tasks_to_run = ['binary_type', 'category', 'application']
    else:
        tasks_to_run = ['category', 'application']

    # Storage: summary_results[task][model_name][feature_set] = metrics
    summary_results = {}

    # --- THE CHAMPIONSHIP LOOP ---
    for task in tasks_to_run:
        print(f"\n{'='*80}")
        print(f"--- STARTING TOURNAMENT FOR TARGET: {task} ---")
        print(f"{'='*80}")

        task_results = [] # List to store flattened results for sorting

        for model_name, model_inst in MODELS.items():
            print(f"\n--- Evaluating Contender: {model_name} ---")

            for set_name, cols in feature_sets.items():
                if not cols: continue

                # Run the task
                metrics = run_classification_task(df, task, set_name, cols, model_name, model_inst)

                # Save results flatly for easy sorting
                metrics['feature_set'] = set_name
                task_results.append(metrics)

        summary_results[task] = task_results

    # --- Final Summary Report ---
    print(f"\n{'='*80}")
    print(f"--- FINAL CHAMPIONSHIP STANDINGS ({EXPERIMENT_MODE}) ---")
    print(f"{'='*80}\n")

    for task, results_list in summary_results.items():
        print(f"--- Target: {task} ---")

        # Sort by Accuracy (Desc)
        sorted_results = sorted(results_list, key=lambda x: x['accuracy'], reverse=True)

        # Define Header
        # M = Model, FS = Feature Set, Acc = Accuracy, F1 = F1-Score, T = Time
        header = f" {'Model':<15} | {'Feature Set':<25} | {'Acc %':<8} | {'F1 %':<8} | {'Time (s)':<8}"
        print(header)
        print("-" * len(header))

        for res in sorted_results:
            print(f" {res['model_name']:<15} | {res['feature_set']:<25} | "
                  f"{res['accuracy']*100:6.2f} | {res['f1_score']*100:6.2f} | "
                  f"{res['time']:6.2f}")
        print("\n")

    print("--- Championship Finished ---")

if __name__ == "__main__":
    if not os.path.exists("/content/drive/MyDrive"):
        print("Please mount your Google Drive first!")
    else:
        main()

--- Initializing PESV v3 Championship (Multi-Model) ---
All libraries imported successfully.

--- Loading Data for Mode: FULL ---
Loaded dataset with shape: (9542, 208)
Found 128 Alpha'' features.
Found 40 Delta features.
Found 36 Gamma' features.

--- STARTING TOURNAMENT FOR TARGET: binary_type ---

--- Evaluating Contender: Random Forest ---

Running: Random Forest | Target: binary_type | Features: Alpha'' (α'') only
 > Acc: 0.9345 | F1: 0.9340 | Time: 1.67s

Running: Random Forest | Target: binary_type | Features: Delta (δ) only
 > Acc: 0.9623 | F1: 0.9619 | Time: 1.13s

Running: Random Forest | Target: binary_type | Features: Gamma' (γ') only
 > Acc: 0.9497 | F1: 0.9489 | Time: 1.17s

Running: Random Forest | Target: binary_type | Features: Alpha'' + Delta
 > Acc: 0.9644 | F1: 0.9644 | Time: 1.70s

Running: Random Forest | Target: binary_type | Features: Alpha'' + Gamma'
 > Acc: 0.9665 | F1: 0.9665 | Time: 3.00s

Running: Random Forest | Target: binary_type | Features: Delta + Gamm

In [ ]:
# --- PESV v3 "Championship" with Hyperparameter Tuning ---
#
# Models: Random Forest vs. XGBoost
# Features: Grid Search for best parameters
# Metrics: Accuracy, Weighted F1, MACRO F1 (Crucial for imbalance)

print("--- Initializing PESV v3 Tuned Championship ---")

import pandas as pd
import numpy as np
import time
import os
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Try importing XGBoost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    print("WARNING: XGBoost not installed. Skipping XGBoost.")
    HAS_XGB = False

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- PART 1: Configuration ---

EXPERIMENT_MODE = "VPN_ONLY"
BASE_PATH = "/content/drive/MyDrive/1 Skripsi/"
FINAL_PESV_FILE = os.path.join(BASE_PATH, "VPNOnly-final_PESV_dataset_v3.csv")
# FINAL_PESV_FILE = os.path.join(BASE_PATH, "final_PESV_dataset_v3.csv") # For FULL mode

TEST_SET_SIZE = 0.2
RANDOM_STATE = 42
CV_FOLDS = 3  # Number of cross-validation folds
N_ITER_SEARCH = 10 # How many random parameter combinations to try per model

# --- PART 2: Model & Parameter Definitions ---

# We define a dictionary of models and their hyperparameter grids
# Note: Parameters must be prefixed with 'classifier__' because they are inside a Pipeline

MODEL_CONFIGS = {
    "Random Forest": {
        "model": RandomForestClassifier(random_state=RANDOM_STATE, class_weight="balanced"),
        "params": {
            "classifier__n_estimators": [100, 200, 300],
            "classifier__max_depth": [None, 10, 20, 30],
            "classifier__min_samples_split": [2, 5, 10],
            "classifier__criterion": ["gini", "entropy"]
        }
    }
}

if HAS_XGB:
    MODEL_CONFIGS["XGBoost"] = {
        "model": XGBClassifier(random_state=RANDOM_STATE, eval_metric='mlogloss'),
        "params": {
            "classifier__n_estimators": [100, 200, 300],
            "classifier__learning_rate": [0.01, 0.1, 0.2],
            "classifier__max_depth": [3, 6, 10],
            "classifier__subsample": [0.8, 1.0]
        }
    }

# --- PART 3: Data Loading Helper ---

def load_data(experiment_mode):
    print(f"\n--- Loading Data for Mode: {experiment_mode} ---")
    if not os.path.exists(FINAL_PESV_FILE):
        print(f"FATAL ERROR: Could not find dataset at '{FINAL_PESV_FILE}'")
        return None, None

    df_full = pd.read_csv(FINAL_PESV_FILE)

    if experiment_mode == "FULL":
        df = df_full
    elif experiment_mode == "VPN_ONLY":
        df = df_full[df_full['binary_type'] == 'VPN'].copy()
    elif experiment_mode == "NONVPN_ONLY":
        df = df_full[df_full['binary_type'] == 'NonVPN'].copy()
    else:
        print(f"ERROR: Unknown mode {experiment_mode}")
        return None, None

    if df.empty: return None, None

    # Define Feature Columns
    all_cols = set(df.columns)
    alpha_cols = sorted([c for c in all_cols if c.startswith('alpha_pp_')])
    delta_cols = sorted([c for c in all_cols if c.startswith(('c2s_', 's2c_', 'flow_', 'total_'))])
    gamma_cols = sorted([c for c in all_cols if c.startswith('burst_')])

    feature_sets = {
        "Alpha'' (α'') only": alpha_cols,
        "Delta (δ) only": delta_cols,
        "Gamma' (γ') only": gamma_cols,
        "Alpha'' + Delta": alpha_cols + delta_cols,
        "Alpha'' + Gamma'": alpha_cols + gamma_cols,
        "Delta + Gamma'": delta_cols + gamma_cols,
        "Full (α'' + δ + γ')": alpha_cols + delta_cols + gamma_cols,
    }

    return df, feature_sets

# --- PART 4: Tuned Classification Task ---

def run_tuned_classification(df, target_label, feature_set_name, feature_cols, model_name, config):
    """
    Runs RandomizedSearchCV to find best params, then evaluates on Test set.
    """
    print(f" > Training {model_name} on {feature_set_name} ({len(feature_cols)} feats)...")

    X = df[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    y = df[target_label]

    # Encode labels (Required for XGBoost)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_names = [str(c) for c in le.classes_]

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=TEST_SET_SIZE, random_state=RANDOM_STATE, stratify=y_encoded
    )

    # Pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', config["model"])
    ])

    # Hyperparameter Tuning (RandomizedSearchCV)
    # We use f1_macro for scoring because of the class imbalance (Skype vs Netflix)
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=config["params"],
        n_iter=N_ITER_SEARCH,
        scoring='f1_macro',
        cv=CV_FOLDS,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0
    )

    start_time = time.time()
    search.fit(X_train, y_train)
    end_time = time.time()

    best_model = search.best_estimator_
    best_params = search.best_params_

    # Predict
    y_pred = best_model.predict(X_test)

    # Metrics extraction
    report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

    metrics = {
        'model': model_name,
        'feature_set': feature_set_name,
        'accuracy': report['accuracy'],

        # Weighted Metrics (Biased towards Skype/Majority)
        'f1_weighted': report['weighted avg']['f1-score'],
        'prec_weighted': report['weighted avg']['precision'],
        'rec_weighted': report['weighted avg']['recall'],

        # Macro Metrics (Treats Netflix same as Skype - Good for Imbalance)
        'f1_macro': report['macro avg']['f1-score'],
        'prec_macro': report['macro avg']['precision'],
        'rec_macro': report['macro avg']['recall'],

        'time': end_time - start_time,
        'best_params': str(best_params).replace("classifier__", "") # Clean string
    }

    return metrics

# --- PART 5: Main Loop ---

def main():
    df, feature_sets = load_data(EXPERIMENT_MODE)
    if df is None: return

    tasks = ['category', 'application']
    summary_results = {}

    for task in tasks:
        print(f"\n{'='*80}")
        print(f"--- TUNING AND EVALUATING TARGET: {task} ---")
        print(f"{'='*80}")

        task_metrics = []

        for model_name, config in MODEL_CONFIGS.items():
            for fs_name, fs_cols in feature_sets.items():
                if not fs_cols: continue

                # Run Logic
                m = run_tuned_classification(df, task, fs_name, fs_cols, model_name, config)
                task_metrics.append(m)

        summary_results[task] = task_metrics

    # ... (Keep the training loops above) ...

    # --- Final Reporting (REPLACED) ---
    print(f"\n{'='*100}")
    print(f"--- FINAL TUNED CHAMPIONSHIP RESULTS ({EXPERIMENT_MODE}) ---")
    print(f"{'='*100}\n")

    for task, results in summary_results.items():
        print(f"--- Target: {task} ---")

        # Sort by Macro F1
        sorted_res = sorted(results, key=lambda x: x['f1_macro'], reverse=True)

        # 1. Print the Summary Table
        print(f"{'Model':<15} | {'Feature Set':<22} | {'Acc':<6} | {'F1(W)':<6} | {'F1(Mac)':<8} | {'Time':<5}")
        print("-" * 80)

        for r in sorted_res:
            print(f"{r['model']:<15} | {r['feature_set']:<22} | "
                  f"{r['accuracy']:.4f} | {r['f1_weighted']:.4f} | {r['f1_macro']:.4f}   | "
                  f"{r['time']:<5.1f}")
        print("\n")

        # 2. Print FULL Parameters for Top 3 Models
        print(f"--- 🏆 Top 3 Models for {task} (Full Parameters) ---")
        for i, r in enumerate(sorted_res[:3]):
            print(f"{i+1}. {r['model']} [{r['feature_set']}]")
            print(f"   Accuracy: {r['accuracy']:.4f} | Macro F1: {r['f1_macro']:.4f}")
            print(f"   Best Params: {r['best_params']}") # Prints the full string
            print("-" * 50)
        print("\n\n")

if __name__ == "__main__":
    main()

--- Initializing PESV v3 Tuned Championship ---

--- Loading Data for Mode: VPN_ONLY ---

--- TUNING AND EVALUATING TARGET: category ---
 > Training Random Forest on Alpha'' (α'') only (128 feats)...
 > Training Random Forest on Delta (δ) only (40 feats)...
 > Training Random Forest on Gamma' (γ') only (36 feats)...
 > Training Random Forest on Alpha'' + Delta (168 feats)...
 > Training Random Forest on Alpha'' + Gamma' (164 feats)...
 > Training Random Forest on Delta + Gamma' (76 feats)...
 > Training Random Forest on Full (α'' + δ + γ') (204 feats)...
 > Training XGBoost on Alpha'' (α'') only (128 feats)...
 > Training XGBoost on Delta (δ) only (40 feats)...
 > Training XGBoost on Gamma' (γ') only (36 feats)...
 > Training XGBoost on Alpha'' + Delta (168 feats)...
 > Training XGBoost on Alpha'' + Gamma' (164 feats)...
 > Training XGBoost on Delta + Gamma' (76 feats)...
 > Training XGBoost on Full (α'' + δ + γ') (204 feats)...

--- TUNING AND EVALUATING TARGET: application ---
 > Tra

In [ ]:
# --- PESV v3 "Championship" with Hyperparameter Tuning ---
#
# Models: Random Forest vs. XGBoost
# Features: Grid Search for best parameters
# Metrics: Accuracy, Weighted F1, MACRO F1 (Crucial for imbalance)

print("--- Initializing PESV v3 Tuned Championship ---")

import pandas as pd
import numpy as np
import time
import os
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Try importing XGBoost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    print("WARNING: XGBoost not installed. Skipping XGBoost.")
    HAS_XGB = False

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- PART 1: Configuration ---

EXPERIMENT_MODE = "FULL"
BASE_PATH = "/content/drive/MyDrive/1 Skripsi/"
FINAL_PESV_FILE = os.path.join(BASE_PATH, "final_PESV_dataset_v3.csv")
# FINAL_PESV_FILE = os.path.join(BASE_PATH, "final_PESV_dataset_v3.csv") # For FULL mode

TEST_SET_SIZE = 0.2
RANDOM_STATE = 42
CV_FOLDS = 3  # Number of cross-validation folds
N_ITER_SEARCH = 10 # How many random parameter combinations to try per model

# --- PART 2: Model & Parameter Definitions ---

# We define a dictionary of models and their hyperparameter grids
# Note: Parameters must be prefixed with 'classifier__' because they are inside a Pipeline

MODEL_CONFIGS = {
    "Random Forest": {
        "model": RandomForestClassifier(random_state=RANDOM_STATE, class_weight="balanced"),
        "params": {
            "classifier__n_estimators": [100, 200, 300],
            "classifier__max_depth": [None, 10, 20, 30],
            "classifier__min_samples_split": [2, 5, 10],
            "classifier__criterion": ["gini", "entropy"]
        }
    }
}

if HAS_XGB:
    MODEL_CONFIGS["XGBoost"] = {
        "model": XGBClassifier(random_state=RANDOM_STATE, eval_metric='mlogloss'),
        "params": {
            "classifier__n_estimators": [100, 200, 300],
            "classifier__learning_rate": [0.01, 0.1, 0.2],
            "classifier__max_depth": [3, 6, 10],
            "classifier__subsample": [0.8, 1.0]
        }
    }

# --- PART 3: Data Loading Helper ---

def load_data(experiment_mode):
    print(f"\n--- Loading Data for Mode: {experiment_mode} ---")
    if not os.path.exists(FINAL_PESV_FILE):
        print(f"FATAL ERROR: Could not find dataset at '{FINAL_PESV_FILE}'")
        return None, None

    df_full = pd.read_csv(FINAL_PESV_FILE)

    if experiment_mode == "FULL":
        df = df_full
    elif experiment_mode == "VPN_ONLY":
        df = df_full[df_full['binary_type'] == 'VPN'].copy()
    elif experiment_mode == "NONVPN_ONLY":
        df = df_full[df_full['binary_type'] == 'NonVPN'].copy()
    else:
        print(f"ERROR: Unknown mode {experiment_mode}")
        return None, None

    if df.empty: return None, None

    # Define Feature Columns
    all_cols = set(df.columns)
    alpha_cols = sorted([c for c in all_cols if c.startswith('alpha_pp_')])
    delta_cols = sorted([c for c in all_cols if c.startswith(('c2s_', 's2c_', 'flow_', 'total_'))])
    gamma_cols = sorted([c for c in all_cols if c.startswith('burst_')])

    feature_sets = {
        "Alpha'' (α'') only": alpha_cols,
        "Delta (δ) only": delta_cols,
        "Gamma' (γ') only": gamma_cols,
        "Alpha'' + Delta": alpha_cols + delta_cols,
        "Alpha'' + Gamma'": alpha_cols + gamma_cols,
        "Delta + Gamma'": delta_cols + gamma_cols,
        "Full (α'' + δ + γ')": alpha_cols + delta_cols + gamma_cols,
    }

    return df, feature_sets

# --- PART 4: Tuned Classification Task ---

def run_tuned_classification(df, target_label, feature_set_name, feature_cols, model_name, config):
    """
    Runs RandomizedSearchCV to find best params, then evaluates on Test set.
    """
    print(f" > Training {model_name} on {feature_set_name} ({len(feature_cols)} feats)...")

    X = df[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    y = df[target_label]

    # Encode labels (Required for XGBoost)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    class_names = [str(c) for c in le.classes_]

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=TEST_SET_SIZE, random_state=RANDOM_STATE, stratify=y_encoded
    )

    # Pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', config["model"])
    ])

    # Hyperparameter Tuning (RandomizedSearchCV)
    # We use f1_macro for scoring because of the class imbalance (Skype vs Netflix)
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=config["params"],
        n_iter=N_ITER_SEARCH,
        scoring='f1_macro',
        cv=CV_FOLDS,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=0
    )

    start_time = time.time()
    search.fit(X_train, y_train)
    end_time = time.time()

    best_model = search.best_estimator_
    best_params = search.best_params_

    # Predict
    y_pred = best_model.predict(X_test)

    # Metrics extraction
    report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

    metrics = {
        'model': model_name,
        'feature_set': feature_set_name,
        'accuracy': report['accuracy'],

        # Weighted Metrics (Biased towards Skype/Majority)
        'f1_weighted': report['weighted avg']['f1-score'],
        'prec_weighted': report['weighted avg']['precision'],
        'rec_weighted': report['weighted avg']['recall'],

        # Macro Metrics (Treats Netflix same as Skype - Good for Imbalance)
        'f1_macro': report['macro avg']['f1-score'],
        'prec_macro': report['macro avg']['precision'],
        'rec_macro': report['macro avg']['recall'],

        'time': end_time - start_time,
        'best_params': str(best_params).replace("classifier__", "") # Clean string
    }

    return metrics

# --- PART 5: Main Loop ---

def main():
    df, feature_sets = load_data(EXPERIMENT_MODE)
    if df is None: return

    tasks = ['binary_type', 'category', 'application']
    summary_results = {}

    for task in tasks:
        print(f"\n{'='*80}")
        print(f"--- TUNING AND EVALUATING TARGET: {task} ---")
        print(f"{'='*80}")

        task_metrics = []

        for model_name, config in MODEL_CONFIGS.items():
            for fs_name, fs_cols in feature_sets.items():
                if not fs_cols: continue

                # Run Logic
                m = run_tuned_classification(df, task, fs_name, fs_cols, model_name, config)
                task_metrics.append(m)

        summary_results[task] = task_metrics

    # ... (Keep the training loops above) ...

    # --- Final Reporting (REPLACED) ---
    print(f"\n{'='*100}")
    print(f"--- FINAL TUNED CHAMPIONSHIP RESULTS ({EXPERIMENT_MODE}) ---")
    print(f"{'='*100}\n")

    for task, results in summary_results.items():
        print(f"--- Target: {task} ---")

        # Sort by Macro F1
        sorted_res = sorted(results, key=lambda x: x['f1_macro'], reverse=True)

        # 1. Print the Summary Table
        print(f"{'Model':<15} | {'Feature Set':<22} | {'Acc':<6} | {'F1(W)':<6} | {'F1(Mac)':<8} | {'Time':<5}")
        print("-" * 80)

        for r in sorted_res:
            print(f"{r['model']:<15} | {r['feature_set']:<22} | "
                  f"{r['accuracy']:.4f} | {r['f1_weighted']:.4f} | {r['f1_macro']:.4f}   | "
                  f"{r['time']:<5.1f}")
        print("\n")

        # 2. Print FULL Parameters for Top 3 Models
        print(f"--- 🏆 Top 3 Models for {task} (Full Parameters) ---")
        for i, r in enumerate(sorted_res[:3]):
            print(f"{i+1}. {r['model']} [{r['feature_set']}]")
            print(f"   Accuracy: {r['accuracy']:.4f} | Macro F1: {r['f1_macro']:.4f}")
            print(f"   Best Params: {r['best_params']}") # Prints the full string
            print("-" * 50)
        print("\n\n")

if __name__ == "__main__":
    main()

--- Initializing PESV v3 Tuned Championship ---

--- Loading Data for Mode: FULL ---

--- TUNING AND EVALUATING TARGET: binary_type ---
 > Training Random Forest on Alpha'' (α'') only (128 feats)...
 > Training Random Forest on Delta (δ) only (40 feats)...
 > Training Random Forest on Gamma' (γ') only (36 feats)...
 > Training Random Forest on Alpha'' + Delta (168 feats)...
 > Training Random Forest on Alpha'' + Gamma' (164 feats)...
 > Training Random Forest on Delta + Gamma' (76 feats)...
 > Training Random Forest on Full (α'' + δ + γ') (204 feats)...
 > Training XGBoost on Alpha'' (α'') only (128 feats)...
 > Training XGBoost on Delta (δ) only (40 feats)...
 > Training XGBoost on Gamma' (γ') only (36 feats)...
 > Training XGBoost on Alpha'' + Delta (168 feats)...
 > Training XGBoost on Alpha'' + Gamma' (164 feats)...
 > Training XGBoost on Delta + Gamma' (76 feats)...
 > Training XGBoost on Full (α'' + δ + γ') (204 feats)...

--- TUNING AND EVALUATING TARGET: category ---
 > Trainin